In [1]:
%load_ext autoreload
from __future__ import print_function, division

In [2]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter


#from mmd_grud_utils import *

In [3]:
DATA_FILEPATH     = '../data/20%/all_hourly_data_7000.h5'
RAW_DATA_FILEPATH = '../data/20%_arr/all_hourly_data_7000.h5'
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)
torch.manual_seed(SEED)

In [4]:
class DictDist():
    def __init__(self, dict_of_rvs): self.dict_of_rvs = dict_of_rvs
    def rvs(self, n):
        a = {k: v.rvs(n) for k, v in self.dict_of_rvs.items()}
        out = []
        for i in range(n): out.append({k: vs[i] for k, vs in a.items()})
        return out
    
class Choice():
    def __init__(self, options): self.options = options
    def rvs(self, n): return [self.options[i] for i in ss.randint(0, len(self.options)).rvs(n)]

In [5]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')
data_full_raw  = pd.read_hdf(RAW_DATA_FILEPATH, 'vitals_labs') 
statics        = pd.read_hdf(DATA_FILEPATH, 'patients')

CPU times: user 4.3 s, sys: 2.9 s, total: 7.2 s
Wall time: 8.13 s


In [6]:
data_full_lvl2.head()

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   

LEVEL2                                 white blood cell count urine           \
Aggregation Function                                          count mean std   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                                 0.0  NaN NaN   
                              1                                 0.0  NaN NaN   
                              2                                 0.0  NaN NaN   
                              3                                 0.0  NaN NaN   
                              4                                 0.0  NaN NaN   

LEVEL2                                    ph                 ph urine           
Aggregation Function                   count  mean       std    count mean std  
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0          9.0  7.40  0.147733      1.0  5.0 NaN  
                              1          0.0   NaN       NaN      0.0  NaN NaN  
                              2          3.0  7.26  0.000000      0.0  NaN NaN  
                              3          0.0   NaN       NaN      0.0  NaN NaN  
                              4          0.0   NaN       NaN      0.0  NaN NaN  

[5 rows x 312 columns]

In [7]:
#data_full_raw.head(20)

In [8]:
statics.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,...,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,...,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0,0,0,1,0,145
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,...,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0,0,0,1,0,40
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,...,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0,0,0,1,0,88
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,...,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1,1,1,1,0,127
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,...,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0,0,0,1,0,38


In [9]:
def simple_imputer(df):
    idx = pd.IndexSlice
    df = df.copy()
    if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
    
    df_out = df.loc[:, idx[:, ['mean', 'count']]]
    icustay_means = df_out.loc[:, idx[:, 'mean']].groupby(ID_COLS).mean()
    
    df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(
        method='ffill'
    ).groupby(ID_COLS).fillna(icustay_means).fillna(0)
    
    df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
    df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)
    
    is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
    hours_of_absence = is_absent.cumsum()
    time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
    time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

    df_out = pd.concat((df_out, time_since_measured), axis=1)
    df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
    
    df_out.sort_index(axis=1, inplace=True)
    return df_out

In [10]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)

lvl2, raw = [df[
    (df.index.get_level_values('icustay_id').isin(set(Ys.index.get_level_values('icustay_id')))) &
    (df.index.get_level_values('hours_in') < WINDOW_SIZE)
] for df in (data_full_lvl2, data_full_raw)]

#raw.columns = raw.columns.droplevel(level=['label', 'LEVEL1', 'LEVEL2'])

train_frac, dev_frac, test_frac = 0.7, 0.1, 0.2
lvl2_subj_idx, raw_subj_idx, Ys_subj_idx = [df.index.get_level_values('subject_id') for df in (lvl2, raw, Ys)]
lvl2_subjects = set(lvl2_subj_idx)
assert lvl2_subjects == set(Ys_subj_idx), "Subject ID pools differ!"
#assert lvl2_subjects == set(raw_subj_idx), "Subject ID pools differ!"

np.random.seed(SEED)
subjects, N = np.random.permutation(list(lvl2_subjects)), len(lvl2_subjects)
N_train, N_dev, N_test = int(train_frac * N), int(dev_frac * N), int(test_frac * N)
train_subj = subjects[:N_train]
dev_subj   = subjects[N_train:N_train + N_dev]
test_subj  = subjects[N_train+N_dev:]

[(lvl2_train, lvl2_dev, lvl2_test), (raw_train, raw_dev, raw_test), (Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (lvl2, raw, Ys)
]

idx = pd.IndexSlice
lvl2_means, lvl2_stds = lvl2_train.loc[:, idx[:,'mean']].mean(axis=0), lvl2_train.loc[:, idx[:,'mean']].std(axis=0)
#raw_means, raw_stds = raw_train.loc[:, idx[:,'mean']].mean(axis=0), raw_train.loc[:, idx[:,'mean']].std(axis=0)

lvl2_train.loc[:, idx[:,'mean']] = (lvl2_train.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_dev.loc[:, idx[:,'mean']] = (lvl2_dev.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds
lvl2_test.loc[:, idx[:,'mean']] = (lvl2_test.loc[:, idx[:,'mean']] - lvl2_means)/lvl2_stds

#raw_train.loc[:, idx[:,'mean']] = (raw_train.loc[:, idx[:,'mean']] - raw_means)/raw_stds
#raw_dev.loc[:, idx[:,'mean']] = (raw_dev.loc[:, idx[:,'mean']] - raw_means)/raw_stds
#raw_test.loc[:, idx[:,'mean']] = (raw_test.loc[:, idx[:,'mean']] - raw_means)/raw_stds

/Users/mengxuan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [11]:

#raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test = [
#    simple_imputer(df) for df in (raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test)
#]
#raw_flat_train, raw_flat_dev, raw_flat_test, lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test = [
#    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
#        raw_train, raw_dev, raw_test, lvl2_train, lvl2_dev, lvl2_test
#    )
#]

#for df in lvl2_train, lvl2_dev, lvl2_test, raw_train, raw_dev, raw_test: assert not df.isnull().any().any()

In [12]:
lvl2_train, lvl2_dev, lvl2_test = [
    simple_imputer(df) for df in (lvl2_train, lvl2_dev, lvl2_test)
]
lvl2_flat_train, lvl2_flat_dev, lvl2_flat_test = [
    df.pivot_table(index=['subject_id', 'hadm_id', 'icustay_id'], columns=['hours_in']) for df in (
        lvl2_train, lvl2_dev, lvl2_test
    )
]

for df in lvl2_train, lvl2_dev, lvl2_test: assert not df.isnull().any().any()


/Users/mengxuan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [13]:
Ys = statics[statics.max_hours > WINDOW_SIZE + GAP_TIME][['mort_hosp', 'mort_icu', 'los_icu']]
Ys['los_3'] = Ys['los_icu'] > 3
Ys['los_7'] = Ys['los_icu'] > 7
Ys.drop(columns=['los_icu'], inplace=True)
Ys.astype(float)
[(Ys_train, Ys_dev, Ys_test)] = [
    [df[df.index.get_level_values('subject_id').isin(s)] for s in (train_subj, dev_subj, test_subj)] \
    for df in (Ys,)
]

### Task Prediction

#### Hyperparams

In [14]:
N = 10

GRU_D_dist = DictDist({
    'cell_size': ss.randint(50, 75),
    'hidden_size': ss.randint(65, 95), 
    'learning_rate': ss.uniform(2e-3, 1e-1),
    'num_epochs': ss.randint(15, 150),
    'patience': ss.randint(3, 7),
    'batch_size': ss.randint(35, 65),
    'early_stop_frac': ss.uniform(0.05, 0.1),
    'seed': ss.randint(1, 10000),
})
np.random.seed(SEED)
GRU_D_hyperparams_list = GRU_D_dist.rvs(N)

#with open('../scratch/mmd/baselines_gru-d.pkl', mode='rb') as f: results = pickle.load(f)

### GRU-D Model

In [15]:
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter

def to_3D_tensor(df):
    idx = pd.IndexSlice
    return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
def prepare_dataloader(df, Ys, batch_size, shuffle=True):
    """
    dfs = (df_train, df_dev, df_test).
    df_* = (subject, hadm, icustay, hours_in) X (level2, agg fn \ni {mask, mean, time})
    Ys_series = (subject, hadm, icustay) => label.
    """
    X     = torch.from_numpy(to_3D_tensor(df).astype(np.float32))
    #print(X[:5])
    label = torch.from_numpy(Ys.values.astype(np.int64))
    dataset = utils.TensorDataset(X, label)
    
    return utils.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last = True)

class FilterLinear(nn.Module):
    def __init__(self, in_features, out_features, filter_square_matrix, bias=True):
        '''
        filter_square_matrix : filter square matrix, whose each elements is 0 or 1.
        '''
        super(FilterLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        assert in_features > 1 and out_features > 1, "Passing in nonsense sizes"
        
        use_gpu = torch.cuda.is_available()
        self.filter_square_matrix = None
        if use_gpu: self.filter_square_matrix = Variable(filter_square_matrix.cuda(), requires_grad=False)
        else:       self.filter_square_matrix = Variable(filter_square_matrix, requires_grad=False)
        
        self.weight = Parameter(torch.Tensor(out_features, in_features))

        if bias: self.bias = Parameter(torch.Tensor(out_features))
        else:    self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None: self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        return F.linear(
            x,
            self.filter_square_matrix.mul(self.weight),
            self.bias
        )

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'
        


In [16]:

class GRUD(nn.Module):
    def __init__(self, input_size, cell_size, hidden_size, X_mean, batch_size = 0, output_last = False):
        """
        With minor modifications from https://github.com/zhiyongc/GRU-D/

        Recurrent Neural Networks for Multivariate Times Series with Missing Values
        GRU-D: GRU exploit two representations of informative missingness patterns, i.e., masking and time interval.
        cell_size is the size of cell_state.
        
        Implemented based on the paper: 
        @article{che2018recurrent,
          title={Recurrent neural networks for multivariate time series with missing values},
          author={Che, Zhengping and Purushotham, Sanjay and Cho, Kyunghyun and Sontag, David and Liu, Yan},
          journal={Scientific reports},
          volume={8},
          number={1},
          pages={6085},
          year={2018},
          publisher={Nature Publishing Group}
        }
        
        GRU-D:
            input_size: variable dimension of each time
            hidden_size: dimension of hidden_state
            mask_size: dimension of masking vector
            X_mean: the mean of the historical input data
        """
        
        super(GRUD, self).__init__()
        
        self.hidden_size = hidden_size
        self.delta_size = input_size
        self.mask_size = input_size
        
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            self.identity = torch.eye(input_size).cuda()
            self.zeros = Variable(torch.zeros(batch_size, input_size).cuda())
            self.zeros_h = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            self.X_mean = Variable(torch.Tensor(X_mean).cuda())
        else:
            self.identity = torch.eye(input_size)
            self.zeros = Variable(torch.zeros(batch_size, input_size))
            self.zeros_h = Variable(torch.zeros(batch_size, self.hidden_size))
            self.X_mean = Variable(torch.Tensor(X_mean))
        
        self.zl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size) # Wz, Uz are part of the same network. the bias is bz
        self.rl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size) # Wr, Ur are part of the same network. the bias is br
        self.hl = nn.Linear(input_size + hidden_size + self.mask_size, hidden_size) # W, U are part of the same network. the bias is b
        
        self.gamma_x_l = FilterLinear(self.delta_size, self.delta_size, self.identity)
        
        self.gamma_h_l = nn.Linear(self.delta_size, self.hidden_size) # this was wrong in available version. remember to raise the issue
        
        self.output_last = output_last
        
        self.fc = nn.Linear(self.hidden_size, 2)
        self.bn= torch.nn.BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True)
        self.drop=nn.Dropout(p=0.5, inplace=False)
        
    def step(self, x, x_last_obsv, x_mean, h, mask, delta):
        """
        Inputs:
            x: input tensor
            x_last_obsv: input tensor with forward fill applied
            x_mean: the mean of each feature
            h: the hidden state of the network
            mask: the mask of whether or not the current value is observed
            delta: the tensor indicating the number of steps since the last time a feature was observed.
            
        Returns:
            h: the updated hidden state of the network
        """
        
        batch_size = x.size()[0]
        dim_size = x.size()[1]
        
        gamma_x_l_delta = self.gamma_x_l(delta)
        delta_x = torch.exp(-torch.max(self.zeros, gamma_x_l_delta)) #exponentiated negative rectifier
        
        gamma_h_l_delta = self.gamma_h_l(delta)
        delta_h = torch.exp(-torch.max(self.zeros_h, gamma_h_l_delta)) #self.zeros became self.zeros_h to accomodate hidden size != input size
        
        x_mean = x_mean.repeat(batch_size, 1)
   
        x = mask * x + (1 - mask) * (delta_x * x_last_obsv + (1 - delta_x) * x_mean)
        x = torch.nan_to_num(x)
        
        h = delta_h * h
       
        combined = torch.cat((x, h, mask), 1)
        z = torch.sigmoid(self.zl(combined)) #sigmoid(W_z*x_t + U_z*h_{t-1} + V_z*m_t + bz)
        r = torch.sigmoid(self.rl(combined)) #sigmoid(W_r*x_t + U_r*h_{t-1} + V_r*m_t + br)
        combined_new = torch.cat((x, r*h, mask), 1)
        h_tilde = torch.tanh(self.hl(combined_new)) #tanh(W*x_t +U(r_t*h_{t-1}) + V*m_t) + b
        h = (1 - z) * h + z * h_tilde
        h = torch.nan_to_num(h)
        return h
    
    def forward(self, X, X_last_obsv, Mask, Delta):
        batch_size = X.size(0)
#         type_size = input.size(1)
        step_size = X.size(1) # num timepoints
        spatial_size = X.size(2) # num features
        
        Hidden_State = self.initHidden(batch_size)
#         X = torch.squeeze(input[:,0,:,:])
#         X_last_obsv = torch.squeeze(input[:,1,:,:])
#         Mask = torch.squeeze(input[:,2,:,:])
#         Delta = torch.squeeze(input[:,3,:,:])
        
        outputs = None
        for i in range(step_size):
            Hidden_State = self.step(
                torch.squeeze(X[:,i:i+1,:], 1),
                torch.squeeze(X_last_obsv[:,i:i+1,:], 1),
                torch.squeeze(self.X_mean[:,i:i+1,:], 1),
                Hidden_State,
                torch.squeeze(Mask[:,i:i+1,:], 1),
                torch.squeeze(Delta[:,i:i+1,:], 1),
            )
            if outputs is None:
                outputs = Hidden_State.unsqueeze(1)
            else:
                outputs = torch.cat((Hidden_State.unsqueeze(1), outputs), 1)
                
        # we want to predict a binary outcome
        #Apply 50% dropout and batch norm here
        self.drop(self.bn(self.fc(Hidden_State)))
        return self.drop(self.bn(self.fc(Hidden_State)))
                
#         if self.output_last:
#             return outputs[:,-1,:]
#         else:
#             return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State

        


In [17]:

def Train_Model(
    model, train_dataloader, valid_dataloader, num_epochs = 300, patience = 3, min_delta = 1e-5, learning_rate=1e-3, batch_size=None
):
    
    print('Model Structure: ', model)
    print('Start Training ... ')
    
    model
    
    if (type(model) == nn.modules.container.Sequential):
        output_last = model[-1].output_last
        print('Output type dermined by the last layer')
    else:
        output_last = model.output_last
        print('Output type dermined by the model')
        
    loss_MSE = torch.nn.MSELoss()
    loss_nll=torch.nn.NLLLoss()
    loss_CEL=torch.nn.CrossEntropyLoss()
    loss_L1 = torch.nn.L1Loss()
    
    learning_rate = 0.001
#     optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate, alpha=0.99)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    use_gpu = False#torch.cuda.is_available()
    
    interval = 100
    losses_train = []
    losses_valid = []
    losses_epochs_train = []
    losses_epochs_valid = []
    
    cur_time = time.time()
    pre_time = time.time()
    
    # Variables for Early Stopping
    is_best_model = 0
    patient_epoch = 0
    for epoch in range(num_epochs):
        
        trained_number = 0
        
        valid_dataloader_iter = iter(valid_dataloader)
        
        losses_epoch_train = []
        losses_epoch_valid = []
        
        for X, labels in train_dataloader:
            X = X.numpy()
         
            mask        = torch.from_numpy(X[:, np.arange(0, X.shape[1], 3), :].astype(np.float32))
            measurement = torch.from_numpy(X[:, np.arange(1, X.shape[1], 3), :].astype(np.float32))
            time_       = torch.from_numpy(X[:, np.arange(2, X.shape[1], 3), :].astype(np.float32))
            
            mask = torch.transpose(mask, 1, 2)
            measurement = torch.transpose(measurement, 1, 2)
            time_ = torch.transpose(time_, 1, 2)
            measurement_last_obsv = measurement            

            assert measurement.size()[0] == batch_size, "Batch Size doesn't match! %s" % str(measurement.size())

            if use_gpu:
                convert_to_cuda=lambda x: Variable(x.cuda())
                X, X_last_obsv, Mask, Delta, labels = map(convert_to_cuda, [measurement, measurement_last_obsv, mask, time_, labels])
            else: 
#                 inputs, labels = Variable(inputs), Variable(labels)
                convert_to_tensor=lambda x: Variable(x)
                X, X_last_obsv, Mask, Delta, labels  = map(convert_to_tensor, [measurement, measurement_last_obsv, mask, time_, labels])
            
            model.zero_grad()

#             outputs = model(inputs)
            prediction=model(X, X_last_obsv, Mask, Delta)
    
#             print(torch.sum(torch.sum(torch.isnan(prediction))))
            
#             print(labels.shape)
#             print(prediction.shape)
            
            if output_last:
                loss_train = loss_CEL(torch.squeeze(prediction), torch.squeeze(labels))
            else:
                full_labels = torch.cat((inputs[:,1:,:], labels), dim = 1)
                loss_train = loss_MSE(outputs, full_labels)
                
      
            losses_train.append(loss_train.data)
            losses_epoch_train.append(loss_train.data)
            
            optimizer.zero_grad()
            
            loss_train.backward()
            
            optimizer.step()
            
             # validation 
            try: 
                X_val, labels_val = next(valid_dataloader_iter)
                X_val = X_val.numpy()
                mask_val        = torch.from_numpy(X_val[:, np.arange(0, X_val.shape[1], 3), :].astype(np.float32))
                measurement_val = torch.from_numpy(X_val[:, np.arange(1, X_val.shape[1], 3), :].astype(np.float32))
                time_val       = torch.from_numpy(X_val[:, np.arange(2, X_val.shape[1], 3), :].astype(np.float32))
            
                mask_val = torch.transpose(mask_val, 1, 2)
                measurement_val = torch.transpose(measurement_val, 1, 2)
                time_val = torch.transpose(time_val, 1, 2)
                measurement_last_obsv_val = measurement_val
            except StopIteration:
                valid_dataloader_iter = iter(valid_dataloader)
                X_val, labels_val = next(valid_dataloader_iter)
                X_val = X_val.numpy()
                mask_val        = torch.from_numpy(X_val[:, np.arange(0, X_val.shape[1], 3), :].astype(np.float32))
                measurement_val = torch.from_numpy(X_val[:, np.arange(1, X_val.shape[1], 3), :].astype(np.float32))
                time_val       = torch.from_numpy(X_val[:, np.arange(2, X_val.shape[1], 3), :].astype(np.float32))
            
                mask_val = torch.transpose(mask_val, 1, 2)
                measurement_val = torch.transpose(measurement_val, 1, 2)
                time_val = torch.transpose(time_val, 1, 2)
                measurement_last_obsv_val = measurement_val
            
            if use_gpu:
                convert_to_cuda=lambda x: Variable(x.cuda())
                X_val, X_last_obsv_val, Mask_val, Delta_val, labels_val = map(convert_to_cuda, [measurement_val, measurement_last_obsv_val, mask_val, time_val, labels_val])
            else: 
#                 inputs, labels = Variable(inputs), Variable(labels)
                convert_to_tensor=lambda x: Variable(x)
                X_val, X_last_obsv_val, Mask_val, Delta_val, labels_val = map(convert_to_tensor, [measurement_val, measurement_last_obsv_val, mask_val, time_val, labels_val])
            
                
            model.zero_grad()
            
#             outputs_val = model(inputs_val)
            prediction_val = model(X_val, X_last_obsv_val, Mask_val, Delta_val)
    
#             print(labels.shape)
#             print(prediction_val.shape)
            
            if output_last:
                loss_valid =loss_CEL(torch.squeeze(prediction_val), torch.squeeze(labels_val))
            else:
                raise NotImplementedError("Should be output last!")
                full_labels_val = torch.cat((inputs_val[:,1:,:], labels_val), dim = 1)
                loss_valid = loss_MSE(outputs_val, full_labels_val)
                
          
            losses_valid.append(loss_valid.data)
            losses_epoch_valid.append(loss_valid.data)
            
#             print(sklearn.metrics.roc_auc_score(labels_val.detach().cpu().numpy(), prediction_val.detach().cpu().numpy()[:,1]))
            
            # output
            trained_number += 1
            
        avg_losses_epoch_train = sum(losses_epoch_train).cpu().numpy() / float(len(losses_epoch_train))
        avg_losses_epoch_valid = sum(losses_epoch_valid).cpu().numpy() / float(len(losses_epoch_valid))
        losses_epochs_train.append(avg_losses_epoch_train)
        losses_epochs_valid.append(avg_losses_epoch_valid)
        
        
        # Early Stopping
        if epoch == 0:
            is_best_model = 1
            best_model = model
            min_loss_epoch_valid = 10000.0
            if avg_losses_epoch_valid < min_loss_epoch_valid:
                min_loss_epoch_valid = avg_losses_epoch_valid
        else:
            if min_loss_epoch_valid - avg_losses_epoch_valid > min_delta:
                is_best_model = 1
                best_model = model
                min_loss_epoch_valid = avg_losses_epoch_valid 
                patient_epoch = 0
            else:
                is_best_model = 0
                patient_epoch += 1
                if patient_epoch >= patience:
                    print('Early Stopped at Epoch:', epoch)
                    break
        
        # Print training parameters
        cur_time = time.time()
        print('Epoch: {}, train_loss: {}, valid_loss: {}, time: {}, best model: {}'.format( \
                    epoch, \
                    np.around(avg_losses_epoch_train, decimals=8),\
                    np.around(avg_losses_epoch_valid, decimals=8),\
                    np.around([cur_time - pre_time] , decimals=2),\
                    is_best_model) )
        pre_time = cur_time
#         if epoch==1:
#             break
                
    return best_model, [losses_train, losses_valid, losses_epochs_train, losses_epochs_valid]



In [18]:

def predict_proba(model, dataloader):
    """
    Input:
        model: GRU-D model
        test_dataloader: containing batches of measurement, measurement_last_obsv, mask, time_, labels
    Returns:
        predictions: size[num_samples, 2]
        labels: size[num_samples]
    """
    model.eval()
    use_gpu = False# torch.cuda.is_available()
    
    probabilities = []
    labels        = []
    ethnicities   = []
    genders       = []
    for X, label in dataloader:
        X = X.numpy()
        mask        = torch.from_numpy(X[:, np.arange(0, X.shape[1], 3), :].astype(np.float32))
        measurement = torch.from_numpy(X[:, np.arange(1, X.shape[1], 3), :].astype(np.float32))
        time_       = torch.from_numpy(X[:, np.arange(2, X.shape[1], 3), :].astype(np.float32))

        mask = torch.transpose(mask, 1, 2)
        measurement = torch.transpose(measurement, 1, 2)
        time_ = torch.transpose(time_, 1, 2)
        measurement_last_obsv = measurement            

        if use_gpu:
            convert_to_cuda=lambda x: Variable(x.cuda())
            X, X_last_obsv, Mask, Delta, label = map(convert_to_cuda, [measurement, measurement_last_obsv, mask, time_, label])
        else: 
#                 inputs, labels = Variable(inputs), Variable(labels)
            convert_to_tensor=lambda x: Variable(x)
            X, X_last_obsv, Mask, Delta, label  = map(convert_to_tensor, [measurement, measurement_last_obsv, mask, time_, label])

        
        prob = model(X, X_last_obsv, Mask, Delta)
        
        probabilities.append(prob.detach().cpu().data.numpy())
        labels.append(label.detach().cpu().data.numpy())

    return probabilities, labels


### GRU-D

In [19]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
results = {}
if model_name not in results: results[model_name] = {}
for t in [
    'mort_icu',
#    'los_3',
#    'mort_hosp',
#    'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass


Running model GRU-D on target mort_icu with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:18: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.84331091, valid_loss: 0.89763973, time: [4.84], best model: 1
Epoch: 1, train_loss: 0.71245279, valid_loss: 0.7457945, time: [4.85], best model: 1
Epoch: 2, train_loss: 0.62439

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.81273672, valid_loss: 0.82484498, time: [5.84], best model: 1
Epoch: 1, train_loss: 0.67443015, valid_loss: 0.6997259, time: [5.84], best model: 1
Epoch: 2, train_loss: 0.57935975, valid_loss: 0.60333959, time: [5.85], best model: 1
Epoch: 3, train_loss: 0.54449102, valid_loss: 0.55756926, time: [5.84], best model: 1
Epoch: 4, train_loss: 0.48519076, valid_loss: 0.49162632, time: [5.84],

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.83784383, valid_loss: 0.86072154, time: [5.43], best model: 1
Epoch: 1, train_loss: 0.72658346, valid_loss: 0.71054977, time: [5.39], best model: 1
Epoch: 2, train_loss: 0.64754613, valid_loss: 0.68277023, time: [5.45], best model: 1
Epoch: 3, train_loss: 0.58676737, valid_loss: 0.63857524, time: [5.38], best model: 1
Epoch: 4, train_loss: 0.54827352, valid_loss: 0.56921519, time: [5.35]

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.82433958, valid_loss: 0.82074817, time: [5.27], best model: 1
Epoch: 1, train_loss: 0.70785518, valid_loss: 0.72600649, time: [5.27], best model: 1
Epoch: 2, train_loss: 0.63754135, valid_loss: 0.66989646, time: [5.26], best model: 1
Epoch: 3, train_loss: 0.57344481, valid_loss: 0.60002256, time: [5.17], best model: 1
Epoch: 4, train_loss: 0.51546132, valid_loss: 0.55774019, time: [5.28]

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.8263754, valid_loss: 0.85246516, time: [5.96], best model: 1
Epoch: 1, train_loss: 0.70505015, valid_loss: 0.71953167, time: [6.08], best model: 1
Epoch: 2, train_loss: 0.62029894, valid_loss: 0.63190479, time: [6.1], best model: 1
Epoch: 3, train_loss: 0.5579735, valid_loss: 0.57665468, time: [6.03], best model: 1
Epoch: 4, train_loss: 0.50096204, valid_loss: 0.50709988, time: [5.96], b

In [20]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
#with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target mort_icu with representation lvl2
0.8000927930714506 0.49750594932299874 0.9336349924585219 0.5217391304347827


In [21]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
if model_name not in results: results[model_name] = {}
for t in [
#    'mort_icu',
    'los_3', 
#    'mort_hosp', 
#    'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass

Running model GRU-D on target los_3 with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:17: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.7801476, valid_loss: 0.79148749, time: [4.85], best model: 1
Epoch: 1, train_loss: 0.72961817, valid_loss: 0.69362465, time: [4.86], best model: 1
Epoch: 2, train_loss: 0.68405

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.80214913, valid_loss: 0.79616845, time: [5.97], best model: 1
Epoch: 1, train_loss: 0.70641667, valid_loss: 0.750244, time: [5.94], best model: 1
Epoch: 2, train_loss: 0.67261498, valid_loss: 0.72656659, time: [5.88], best model: 1
Epoch: 3, train_loss: 0.65241304, valid_loss: 0.72756278, time: [5.86], best model: 0
Epoch: 4, train_loss: 0.62889696, valid_loss: 0.71885716, time: [5.87], 

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.7972761, valid_loss: 0.77739044, time: [5.37], best model: 1
Epoch: 1, train_loss: 0.71415492, valid_loss: 0.74521673, time: [5.36], best model: 1
Epoch: 2, train_loss: 0.68596481, valid_loss: 0.73346268, time: [5.37], best model: 1
Epoch: 3, train_loss: 0.66653188, valid_loss: 0.71381185, time: [5.39], best model: 1
Epoch: 4, train_loss: 0.65617096, valid_loss: 0.69874751, time: [5.38],

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.78710618, valid_loss: 0.80771743, time: [5.24], best model: 1
Epoch: 1, train_loss: 0.71299442, valid_loss: 0.74734311, time: [5.24], best model: 1
Epoch: 2, train_loss: 0.68680936, valid_loss: 0.72515625, time: [5.25], best model: 1
Epoch: 3, train_loss: 0.66745337, valid_loss: 0.70148251, time: [5.25], best model: 1
Epoch: 4, train_loss: 0.65130571, valid_loss: 0.7187673, time: [5.25],

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=286, out_features=78, bias=True)
  (rl): Linear(in_features=286, out_features=78, bias=True)
  (hl): Linear(in_features=286, out_features=78, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=78, bias=True)
  (fc): Linear(in_features=78, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.81091111, valid_loss: 0.80550498, time: [5.75], best model: 1
Epoch: 1, train_loss: 0.70704279, valid_loss: 0.74274261, time: [5.73], best model: 1
Epoch: 2, train_loss: 0.67885896, valid_loss: 0.71880468, time: [5.73], best model: 1
Epoch: 3, train_loss: 0.65182208, valid_loss: 0.71013933, time: [5.93], best model: 1
Epoch: 4, train_loss: 0.64442915, valid_loss: 0.69019709, time: [5.71]

In [22]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

#results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target los_3 with representation lvl2
0.6829013396361199 0.6356465917949119 0.6411201179071481 0.5965202982601492


In [23]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
if model_name not in results: results[model_name] = {}
for t in [
#    'mort_icu',
#    'los_3', 
    'mort_hosp', 
#    'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass

Running model GRU-D on target mort_hosp with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:17: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.81440755, valid_loss: 0.77412806, time: [4.88], best model: 1
Epoch: 1, train_loss: 0.69040926, valid_loss: 0.6817498, time: [4.89], best model: 1
Epoch: 2, train_loss: 0.62892

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.79502716, valid_loss: 0.80783754, time: [5.9], best model: 1
Epoch: 1, train_loss: 0.65518362, valid_loss: 0.70107491, time: [5.9], best model: 1
Epoch: 2, train_loss: 0.58783791, valid_loss: 0.60492082, time: [5.9], best model: 1
Epoch: 3, train_loss: 0.52766373, valid_loss: 0.54618582, time: [5.88], best model: 1
Epoch: 4, train_loss: 0.46994057, valid_loss: 0.49760791, time: [5.9], be

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.81873586, valid_loss: 0.80879171, time: [5.47], best model: 1
Epoch: 1, train_loss: 0.70052439, valid_loss: 0.69570913, time: [5.66], best model: 1
Epoch: 2, train_loss: 0.61955505, valid_loss: 0.64534943, time: [5.47], best model: 1
Epoch: 3, train_loss: 0.55446437, valid_loss: 0.61723551, time: [5.5], best model: 1
Epoch: 4, train_loss: 0.50489207, valid_loss: 0.57743037, time: [5.63],

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.78547597, valid_loss: 0.80813608, time: [5.19], best model: 1
Epoch: 1, train_loss: 0.69134938, valid_loss: 0.70730449, time: [5.28], best model: 1
Epoch: 2, train_loss: 0.60039613, valid_loss: 0.62607623, time: [5.18], best model: 1
Epoch: 3, train_loss: 0.55264539, valid_loss: 0.58576841, time: [5.19], best model: 1
Epoch: 4, train_loss: 0.51368119, valid_loss: 0.58771351, time: [5.18]

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=298, out_features=90, bias=True)
  (rl): Linear(in_features=298, out_features=90, bias=True)
  (hl): Linear(in_features=298, out_features=90, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=90, bias=True)
  (fc): Linear(in_features=90, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.77701597, valid_loss: 0.78637107, time: [7.24], best model: 1
Epoch: 1, train_loss: 0.67120361, valid_loss: 0.68088594, time: [7.21], best model: 1
Epoch: 2, train_loss: 0.58633129, valid_loss: 0.60316985, time: [7.22], best model: 1
Epoch: 3, train_loss: 0.51151346, valid_loss: 0.56029143, time: [7.22], best model: 1
Epoch: 4, train_loss: 0.4720684, valid_loss: 0.51903943, time: [7.2], 

In [24]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
#with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target mort_hosp with representation lvl2
0.8226327445578784 0.5392050965807662 0.900953778429934 0.5229681978798586


In [25]:
%%time
model_name       = 'GRU-D'
hyperparams_list = GRU_D_hyperparams_list
RERUN            = False
if model_name not in results: results[model_name] = {}
for t in [
#    'mort_icu',
#    'los_3', 
#    'mort_hosp', 
     'los_7'
]:
    if t not in results[model_name]: results[model_name][t] = {}
    for n, X_train, X_dev, X_test in (
        ('lvl2', lvl2_train, lvl2_dev, lvl2_test),
#         ('raw', raw_train, raw_dev, raw_test)
    ):
        print("Running model %s on target %s with representation %s" % (model_name, t, n))
        X_mean = np.nanmean(
            to_3D_tensor(
                X_train.loc[:, pd.IndexSlice[:, 'mean']] * 
                np.where((X_train.loc[:, pd.IndexSlice[:, 'mask']] == 1).values, 1, np.NaN)
            ),
            axis=0, keepdims=True
        ).transpose([0, 2, 1])
        base_params = {'X_mean': X_mean, 'output_last': True, 'input_size': X_mean.shape[2]}
    
        if n in results[model_name][t]:
            if not RERUN: 
                print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
                print(results[model_name][t][n])
                continue
            best_s, best_hyperparams = results[model_name][t][n][-1], results[model_name][t][n][1]
            print("Loading best hyperparams", best_hyperparams)
        else:
            best_s, best_hyperparams = -np.Inf, None
            for i, hyperparams in enumerate(hyperparams_list):
                print("On sample %d / %d (hyperparams = %s)" % (i+1, len(hyperparams_list), repr((hyperparams))))

                early_stop_frac,batch_size,seed = [hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
                batch_size = int(batch_size)

                np.random.seed(seed)
                all_train_subjects = list(
                    np.random.permutation(Ys_train.index.get_level_values('subject_id').values)
                )
                N_early_stop        = int(len(all_train_subjects) * early_stop_frac)
                train_subjects      = all_train_subjects[:-N_early_stop]
                early_stop_subjects = all_train_subjects[-N_early_stop:]
                X_train_obs         = X_train[X_train.index.get_level_values('subject_id').isin(train_subjects)]
                Ys_train_obs        = Ys_train[Ys_train.index.get_level_values('subject_id').isin(train_subjects)]

                X_train_early_stop  = X_train[X_train.index.get_level_values('subject_id').isin(early_stop_subjects)]
                Ys_train_early_stop = Ys_train[
                    Ys_train.index.get_level_values('subject_id').isin(early_stop_subjects)
                ]

                train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
                early_stop_dataloader = prepare_dataloader(
                    X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size
                )
                dev_dataloader        = prepare_dataloader(X_dev, Ys_dev[t], batch_size=batch_size)
                test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

                model_hyperparams = copy.copy(base_params)
                model_hyperparams.update(
                    {k: v for k, v in hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
                )
                
                batch_size = int(batch_size)
                model = GRUD(**model_hyperparams)

                best_model, _ = Train_Model(
                    model, train_dataloader, early_stop_dataloader,
                    **{k: v for k, v in hyperparams.items() if k in (
                        'num_epochs', 'patience', 'learning_rate', 'batch_size'
                    )}
                )

                probabilities_dev, labels_dev = predict_proba(best_model, dev_dataloader)
                probabilities_dev = np.concatenate(probabilities_dev)[:, 1]
                labels_dev        = np.concatenate(labels_dev)
                s = roc_auc_score(labels_dev, probabilities_dev)
                if s > best_s:
                    best_s, best_hyperparams = s, hyperparams
                    print("New Best Score: %.2f @ hyperparams = %s" % (100*best_s, repr((best_hyperparams))))
                
        ## Test
        try:
            np.random.seed(seed)
            hyperparams = best_hyperparams # In case I forgot a replace below
            early_stop_frac,batch_size,seed = [best_hyperparams[k] for k in ('early_stop_frac','batch_size','seed')]
            batch_size = int(batch_size)
            X_train_concat, Ys_train_concat = pd.concat((X_train, X_dev)), pd.concat((Ys_train, Ys_dev))

            all_train_subjects = list(np.random.permutation(Ys_train_concat.index.get_level_values('subject_id').values))
            N_early_stop = int(len(all_train_subjects) * early_stop_frac)
            train_subjects, early_stop_subjects = all_train_subjects[:-N_early_stop], all_train_subjects[-N_early_stop:]
            X_train_obs         = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(train_subjects)]
            Ys_train_obs        = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(train_subjects)]

            X_train_early_stop  = X_train_concat[X_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]
            Ys_train_early_stop = Ys_train_concat[Ys_train_concat.index.get_level_values('subject_id').isin(early_stop_subjects)]

            train_dataloader      = prepare_dataloader(X_train_obs, Ys_train_obs[t], batch_size=batch_size)
            early_stop_dataloader = prepare_dataloader(X_train_early_stop, Ys_train_early_stop[t], batch_size=batch_size)
            test_dataloader       = prepare_dataloader(X_test, Ys_test[t], batch_size=batch_size)

            model_hyperparams = copy.copy(base_params)
            model_hyperparams.update(
                {k: v for k, v in best_hyperparams.items() if k in ('cell_size', 'hidden_size', 'batch_size')}
            )
            model = GRUD(**model_hyperparams)

            best_model, (losses_train, losses_early_stop, losses_epochs_train, losses_epochs_early_stop) = Train_Model(
                model, train_dataloader, early_stop_dataloader,
                **{k: v for k, v in best_hyperparams.items() if k in (
                    'num_epochs', 'patience', 'learning_rate', 'batch_size'
                )}
            )

            probabilities_test, labels_test = predict_proba(best_model, test_dataloader)

            y_score = np.concatenate(probabilities_test)[:, 1]
            y_pred  = np.argmax(probabilities_test)
            y_true  = np.concatenate(labels_test)

            auc   = roc_auc_score(y_true, y_score)
            auprc = average_precision_score(y_true, y_score)
            acc   = accuracy_score(y_true, y_pred)
            F1    = f1_score(y_true, y_pred)
            print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
            print(auc, auprc, acc, F1)

            results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
            #with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)
        except: pass

Running model GRU-D on target los_7 with representation lvl2


/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))
<timed exec>:17: RuntimeWarning: Mean of empty slice


On sample 1 / 10 (hyperparams = {'cell_size': 55, 'hidden_size': 66, 'learning_rate': 0.07052195003967596, 'num_epochs': 22, 'patience': 6, 'batch_size': 60, 'early_stop_frac': 0.051828827734419186, 'seed': 9496})
Model Structure:  GRUD(
  (zl): Linear(in_features=274, out_features=66, bias=True)
  (rl): Linear(in_features=274, out_features=66, bias=True)
  (hl): Linear(in_features=274, out_features=66, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=66, bias=True)
  (fc): Linear(in_features=66, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.84703335, valid_loss: 0.84791756, time: [4.92], best model: 1
Epoch: 1, train_loss: 0.74589569, valid_loss: 0.75291448, time: [4.9], best model: 1
Epoch: 2, train_loss: 0.69429

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=285, out_features=77, bias=True)
  (rl): Linear(in_features=285, out_features=77, bias=True)
  (hl): Linear(in_features=285, out_features=77, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=77, bias=True)
  (fc): Linear(in_features=77, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.83941241, valid_loss: 0.82581995, time: [5.94], best model: 1
Epoch: 1, train_loss: 0.71951696, valid_loss: 0.7286773, time: [5.94], best model: 1
Epoch: 2, train_loss: 0.62234969, valid_loss: 0.63077101, time: [5.94], best model: 1
Epoch: 3, train_loss: 0.56269923, valid_loss: 0.56069804, time: [5.96], best model: 1
Epoch: 4, train_loss: 0.53207789, valid_loss: 0.53522124, time: [5.96],

...

Model Structure:  GRUD(
  (zl): Linear(in_features=293, out_features=85, bias=True)
  (rl): Linear(in_features=293, out_features=85, bias=True)
  (hl): Linear(in_features=293, out_features=85, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=85, bias=True)
  (fc): Linear(in_features=85, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.83684845, valid_loss: 0.84202108, time: [5.35], best model: 1
Epoch: 1, train_loss: 0.74635351, valid_loss: 0.74879547, time: [5.31], best model: 1
Epoch: 2, train_loss: 0.68235509, valid_loss: 0.67760686, time: [5.31], best model: 1
Epoch: 3, train_loss: 0.61562922, valid_loss: 0.61870377, time: [5.42], best model: 1
Epoch: 4, train_loss: 0.56388138, valid_loss: 0.58035268, time: [5.45]

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=278, out_features=70, bias=True)
  (rl): Linear(in_features=278, out_features=70, bias=True)
  (hl): Linear(in_features=278, out_features=70, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=70, bias=True)
  (fc): Linear(in_features=70, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.83142525, valid_loss: 0.85539308, time: [5.91], best model: 1
Epoch: 1, train_loss: 0.72610775, valid_loss: 0.74103905, time: [5.65], best model: 1
Epoch: 2, train_loss: 0.68256218, valid_loss: 0.68885364, time: [5.77], best model: 1
Epoch: 3, train_loss: 0.61548965, valid_loss: 0.62352336, time: [5.85], best model: 1
Epoch: 4, train_loss: 0.56684818, valid_loss: 0.58103025, time: [5.4],

/var/folders/mc/tyl8krpx7t5c21z141cz0t2c0000gn/T/ipykernel_31323/1966726343.py:13: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.dstack((df.loc[idx[:,:,:,i], :].values for i in sorted(set(df.index.get_level_values('hours_in')))))


Model Structure:  GRUD(
  (zl): Linear(in_features=279, out_features=71, bias=True)
  (rl): Linear(in_features=279, out_features=71, bias=True)
  (hl): Linear(in_features=279, out_features=71, bias=True)
  (gamma_x_l): FilterLinear(in_features=104, out_features=104, bias=True)
  (gamma_h_l): Linear(in_features=104, out_features=71, bias=True)
  (fc): Linear(in_features=71, out_features=2, bias=True)
  (bn): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.5, inplace=False)
)
Start Training ... 
Output type dermined by the model
Epoch: 0, train_loss: 0.83319187, valid_loss: 0.85630299, time: [6.52], best model: 1
Epoch: 1, train_loss: 0.72707785, valid_loss: 0.7333052, time: [6.5], best model: 1
Epoch: 2, train_loss: 0.64345147, valid_loss: 0.65488089, time: [6.27], best model: 1
Epoch: 3, train_loss: 0.5812159, valid_loss: 0.60430072, time: [6.24], best model: 1
Epoch: 4, train_loss: 0.53294428, valid_loss: 0.54689649, time: [6.32], b

In [26]:
y_score = np.concatenate(probabilities_test)[:, 1]
y_pred  = np.concatenate(probabilities_test).argmax(axis=1)
y_true  = np.concatenate(labels_test)

y_score  = np.nan_to_num(y_score , nan=0, posinf=0)
y_pred  = np.nan_to_num(y_pred , nan=0, posinf=0)
y_true  = np.nan_to_num(y_true , nan=0, posinf=0)

auc   = roc_auc_score(y_true, y_score)
auprc = average_precision_score(y_true, y_score)
acc   = accuracy_score(y_true, y_pred)
F1    = f1_score(y_true, y_pred)
print("Final results for model %s on target %s with representation %s" % (model_name, t, n))
print(auc, auprc, acc, F1)

results[model_name][t][n] = None, best_hyperparams, auc, auprc, acc, F1, best_s
with open('../scratch/mmd/baselines_gru-d.pkl', mode='wb') as f: pickle.dump(results, f)

Final results for model GRU-D on target los_7 with representation lvl2
0.6424193783744345 0.1384030084869607 0.9107946026986506 0.11851851851851852
